## All Import Modules

In [1]:
import yfinance as yf
import datetime
import numpy as np
from ta import momentum, trend
import pandas as pd

## MENUPLIATE INPUT DATA

In [2]:
previes_days = 5
def Main_Dataframe(symbol,previes_days):
    # ------------------------------------ DAY TIME FRAME -----------------------------------------------------------
    ticker = yf.Ticker(symbol)
    day_df = ticker.history(period=f"80d", interval="1d")
    
    # Reset the index and convert the 'Date' column to date format
    day_df.reset_index(inplace=True)
    day_df['Date'] = day_df['Date'].dt.date
    
    # Calculate technical indicators
    day_df['Day_10EMA'] = trend.EMAIndicator(day_df['Close'], window=10).ema_indicator()
    
    day_df['Day_RSI'] = momentum.RSIIndicator(day_df['Close'], window=3).rsi()
    day_df['Day_RSI_EMA'] = trend.EMAIndicator(day_df['Day_RSI'], window=6).ema_indicator()

    # -----------ADD EXTRA INDICATORS HERE --------------
    day_df['Day_Trend'] = np.select(
    [day_df['Day_10EMA'] > day_df['Day_10EMA'].shift(1),
     day_df['Day_10EMA'] < day_df['Day_10EMA'].shift(1)],
    ['Uptrend', 'Downtrend'],'Sideways')
    # Rename columns and drop unnecessary columns
    day_df.rename(columns={'Open': 'Day_Open', 'High': 'Day_High', 'Low': 'Day_Low', 'Close': 'Day_Close'}, inplace=True)
    day_df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Day_10EMA','Day_RSI'], inplace=True)
    day_df.dropna(inplace=True)

    # ------------------------------------ 5 MIN TIME FRAME -----------------------------------------------------------
    # previes_days = 5
    df = ticker.history(period=f"{previes_days}d", interval="5m")
    df.reset_index(inplace=True)
    df['Date'] = df['Datetime'].dt.date
    df['Time'] = df['Datetime'].dt.time
    
    df.drop(columns=['Volume', 'Dividends', 'Stock Splits','Datetime'], inplace=True)
    df.dropna(inplace=True)
    # -----------ADD EXTRA INDICATORS HERE --------------
    df['5EMA'] = trend.EMAIndicator(df['Close'], window=5).ema_indicator()
    df['15EMA'] = trend.EMAIndicator(df['Close'], window=15).ema_indicator()
    df['Candle'] = np.where(df['Close'] > df['Open'], 'Green', 'Red')
    
    df['Prev_5EMA'] = df['5EMA'].shift(1)
    df['Prev_15EMA'] = df['15EMA'].shift(1)

    df['RSI'] = momentum.RSIIndicator(df['Close'], window=6).rsi()
    df['RSI_EMA'] = trend.EMAIndicator(df['RSI'], window=12).ema_indicator()
    
   
    short_ema =trend.EMAIndicator(df["Close"], window=5)
    long_ema = trend.EMAIndicator(df["Close"], window=15)
    
    df["5EMA"] = short_ema.ema_indicator()
    df["15EMA"] = long_ema.ema_indicator()
    
    # Determine the perfect trend based on EMA crossovers
    df["Trend"] = "Sideways"  # Default to Sideways
    df.loc[(df["5EMA"] > df["15EMA"]) & (df["5EMA"].shift(1) <= df["15EMA"].shift(1)), "Trend"] = "Uptrend"
    df.loc[(df["5EMA"] < df["15EMA"]) & (df["5EMA"].shift(1) >= df["15EMA"].shift(1)), "Trend"] = "Downtrend"

    first_candle_high = df.groupby('Date')['High'].first()
    first_candle_low = df.groupby('Date')['Low'].first()
    
    # Create a new column for the first candle's high and low
    df['First_Candle_High'] = df['Date'].map(first_candle_high)
    df['First_Candle_Low'] = df['Date'].map(first_candle_low)

    # ------------------------------------ MERGE DATA FRAME -----------------------------------------------------------
    
    merged_df = day_df.merge(df, on='Date', how='inner')
    merged_df.dropna(inplace=True)
    merged_df['DateTime'] = pd.to_datetime(merged_df['Date'].astype(str) + ' ' + merged_df['Time'].astype(str))
    merged_df.drop(['Date', 'Time'], axis=1, inplace=True)
    return round(merged_df,2)
    # return day_df
    # return df

data = Main_Dataframe("^NSEI",previes_days)
# data = Main_Dataframe("HDFCBANK.BO")
# data.tail(5)

In [3]:
all_columsn = ['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA',
        'Open', 'High', 'Low', 'Close', '5EMA', '15EMA',
       'Prev_5EMA', 'Prev_15EMA', 'RSI', 'RSI_EMA', 'First_Candle_High', 'First_Candle_Low']
day_all_columsn = ['Date', 'Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_RSI_EMA']
# data.iloc[-2]

# Add Signals Columns

In [4]:
def Add_Signals(dfnew):
    df = dfnew.copy()
    df['Signal_Day_RSI'] = np.select([df['Day_RSI_EMA'] <= 30, df['Day_RSI_EMA'] >= 70],['CE', 'PE'],default='0')
    
    df['Signal_RSI'] = np.select([df['RSI_EMA'] <= 70, df['RSI_EMA'] >= 30],['CE', 'PE'],default='0')
    # df['Signal_RSI_CE'] = np.where(df['RSI_EMA'] <= 30, 'CE', '0')
    # df['Signal_RSI_PE'] = np.where(df['RSI_EMA'] >= 70, 'PE', '0')


    ce_5ema = (df['High'].shift(1) < df['5EMA'].shift(1)) & (df['Close'] >= df['5EMA'])
    pe_5ema = (df['Low'].shift(1) > df['5EMA'].shift(1)) & (df['Close'] <= df['5EMA'])
    
    df['Signal_5EMA'] = np.select([ce_5ema, pe_5ema],['CE', 'PE'], default='NONE')

    ce_ema_crossover = (df['5EMA'] > df['15EMA']) & (df['5EMA'].shift(1) <= df['15EMA'].shift(1))
    pe_ema_crossover = (df['5EMA'] < df['15EMA']) & (df['5EMA'].shift(1) >= df['15EMA'].shift(1))
    
    df['Signal_EMA_CROSSOVER'] = np.select([ce_ema_crossover, pe_ema_crossover],['CE', 'PE'], default='0')
    
    # df.drop(columns = all_columsn,inplace= True)
    return df
newdata = Add_Signals(data)
newdata

,Day_Open,Day_High,Day_Low,Day_Close,Day_RSI_EMA,Day_Trend,Open,High,Low,Close,...,RSI,RSI_EMA,Trend,First_Candle_High,First_Candle_Low,DateTime,Signal_Day_RSI,Signal_RSI,Signal_5EMA,Signal_EMA_CROSSOVER
16,19828.45,19875.15,19786.75,19802.00,78.66,Uptrend,19805.90,19812.35,19800.70,19803.25,...,25.58,39.26,Sideways,19851.35,19828.45,2023-11-23 10:35:00,PE,CE,NONE,0
17,19828.45,19875.15,19786.75,19802.00,78.66,Uptrend,19803.10,19811.40,19793.30,19803.05,...,25.47,37.14,Sideways,19851.35,19828.45,2023-11-23 10:40:00,PE,CE,NONE,0
18,19828.45,19875.15,19786.75,19802.00,78.66,Uptrend,19803.00,19813.30,19801.90,19804.50,...,28.00,35.74,Sideways,19851.35,19828.45,2023-11-23 10:45:00,PE,CE,NONE,0
19,19828.45,19875.15,19786.75,19802.00,78.66,Uptrend,19804.25,19810.90,19802.65,19810.25,...,38.02,36.09,Sideways,19851.35,19828.45,2023-11-23 10:50:00,PE,CE,NONE,0
20,19828.45,19875.15,19786.75,19802.00,78.66,Uptrend,19810.20,19815.95,19801.65,19805.95,...,33.80,35.73,Sideways,19851.35,19828.45,2023-11-23 10:55:00,PE,CE,NONE,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,20108.50,20158.70,20015.85,20133.15,87.96,Uptrend,20119.60,20139.75,20116.85,20131.70,...,73.27,65.40,Sideways,20134.20,20108.50,2023-11-30 15:05:00,PE,CE,NONE,0
371,20108.50,20158.70,20015.85,20133.15,87.96,Uptrend,20131.10,20135.70,20126.20,20135.20,...,75.16,66.90,Sideways,20134.20,20108.50,2023-11-30 15:10:00,PE,CE,NONE,0
372,20108.50,20158.70,20015.85,20133.15,87.96,Uptrend,20133.85,20138.10,20127.70,20137.95,...,76.71,68.41,Sideways,20134.20,20108.50,2023-11-30 15:15:00,PE,CE,NONE,0
373,20108.50,20158.70,20015.85,20133.15,87.96,Uptrend,20138.00,20155.60,20137.80,20154.40,...,83.92,70.80,Sideways,20134.20,20108.50,2023-11-30 15:20:00,PE,PE,NONE,0


In [5]:
# newdata[(newdata['Signal_RSI'] == 'PE') & (newdata['Signal_EMA_CROSSOVER'] == 'PE')]
# newdata[(newdata['Signal_EMA_CROSSOVER'] == 'PE')].shape[0]
# newdata[(newdata['Signal_RSI_PE'] == 'PE')].shape[0]

# ENTRY EXIT LOGIC